#### Using HeFESTo lookup table in aspect

In this note book, I'll show how to:

* a. Import a HeFESTo lookup table
* b. Plot This lookup table
* c. Estimate the amount of density change and latent heat generation on a mantle adiabat
* d. convert to a "perplex" lookup table readable by aspect
* e. Maintain a workflow using the lookup table, including how to check your implementation.